<a href="https://colab.research.google.com/github/DebadityaShome/Deep-learning-practice/blob/main/PyTorch/tutorials/basics_ZeroToLinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import numpy as np

## **Creating tensors**

In [2]:
# Number
t1 = torch.tensor(5.)
t1

tensor(5.)

In [3]:
# Vector
t2 = torch.tensor([1., 2., 3, 4])
t2

tensor([1., 2., 3., 4.])

In [5]:
# Matrix (2D-array)
t3 = torch.tensor([[5., 6.], [7, 8], [9, 10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [11]:
# 3D-array
t4 = torch.tensor([
                   [
                    [11, 12, 13],
                    [14, 15, 16]
                   ],
                   [
                    [1, 2, 4],
                    [6, 7, 8.]
                   ]
])

In [13]:
t4.shape

torch.Size([2, 2, 3])

## **Tensor operations and gradients**

In [14]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

In [15]:
x, w, b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [16]:
# Arithmetic operations
y = w*x + b
y

tensor(17., grad_fn=<AddBackward0>)

In [17]:
# Compute derivatives
y.backward()

In [18]:
print("dy/dx = ", x.grad) # Output is None as requires_grad not equal to True
print("dy/dw = ", w.grad)
print("dy/db = ", b.grad)

dy/dx =  None
dy/dw =  tensor(3.)
dy/db =  tensor(1.)


## **Tensor Functions**

In [19]:
# Create a tensor with a fixed value for every element
t6 = torch.full((3, 2), 42)
t6

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [20]:
# Concatenate two tensors with compatible shapes
t7 = torch.cat((t3, t6))
t7

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [26]:
# Change the shape of a tensor
t8 = t7.reshape(3, 2, 2)

# **Linear regression dummy example**

In [28]:
# Input data (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [30]:
# Target data (yield of apples and oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [32]:
# Convert numpy arays to PyTorch tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### **Linear regression from scratch**

In [33]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

In [38]:
def model(x):
    return x @ w.t() + b

In [46]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [47]:
# Generate predictions
preds = model(inputs)

In [48]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(44471.7500, grad_fn=<DivBackward0>)


In [49]:
# Compute gradients
loss.backward()

In [50]:
# Gradients for weights
w.grad

tensor([[-21497.5977, -24427.2266, -14749.1748],
        [-10895.1738, -10372.0293,  -6641.2168]])

In [51]:
# Using the pre-computed gradients to perform one step of Gradient descent
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [54]:
# Let's verify if loss really reduced or not
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(30387.0117, grad_fn=<DivBackward0>)


In [55]:
# Making sure all gradients variables are initialized to zero to avoid accumulation by PyTorch
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [88]:
# Final training loop
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [89]:
# Calculate final loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1.2841, grad_fn=<DivBackward0>)


In [90]:
preds

tensor([[ 57.3309,  70.2446],
        [ 82.4592, 101.4237],
        [117.8568, 131.3490],
        [ 20.8318,  36.5607],
        [102.7539, 120.7464]], grad_fn=<AddBackward0>)

In [91]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])